In [ ]:
import random
urls = [f"https://httpbin.org/delay/{random.randint(1,20)*0.1}" for _ in range(1000)]
url = "https://httpbin.org/delay/1"
payload = {"some": "json"}
tries = [1, 10]

In [ ]:
import time
import requests
import logging
import concurrent

logging.getLogger().setLevel(logging.INFO)


def fetch_url(im_url):
    try:
        resp = requests.get(im_url, payload)
    except Exception as e:
        logging.info("could not fetch {}".format(im_url))
    else:
        return resp.json()


def fetch_all(urls):
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        responses = {executor.submit(fetch_url, row): row for row in urls}
        results.extend([future.result()] for future in concurrent.futures.as_completed(responses))
    return results


if __name__ == "__main__":

    for ntimes in tries:
        start_time = time.time()
        futures = fetch_all(urls[:ntimes])
        end_time = time.time()
        logging.info(f"Fetch {ntimes} urls takes {end_time - start_time} seconds")
        logging.info(f"{len(list(futures))} urls were read successfully")
        logging.info(f"conctent: {list(futures)}")


In [ ]:
import time
import logging
import asyncio
from aiohttp import ClientSession, ClientResponseError
# --------------only in jupyter notebook--------------
import nest_asyncio
nest_asyncio.apply()

logging.getLogger().setLevel(logging.INFO)


async def fetch(session, url):
    try:
        async with session.get(url, timeout=15, json=payload) as response:
            resp = await response.json()
    except ClientResponseError as e:
        logging.warning(e.code)
    except asyncio.TimeoutError:
        logging.warning("Timeout")
    except Exception as e:
        logging.warning(e)
    else:
        return resp
    return


async def fetch_async(urls):
    # please use url by your choice
    # url = "*** some image url ****"
    tasks = []
    # try to use one client session
    async with ClientSession() as session:
        for url in urls:
            task = asyncio.ensure_future(fetch(session, url))
            tasks.append(task)
        # await response outside the for loop
        responses = await asyncio.gather(*tasks)
    return responses


if __name__ == "__main__":
    for ntimes in tries:
        start_time = time.time()
        futures = asyncio.run(fetch_async(urls[:ntimes]))
        end_time = time.time()
        logging.info(f"Fetch {ntimes} urls takes {end_time - start_time} seconds")
        logging.info(f"{len(futures)} urls were read successfully")
        logging.info(f"conctent: {list(futures)}")


In [ ]:
import requests
from requests.sessions import Session
import time
from concurrent.futures import ThreadPoolExecutor
from threading import local

def get_session() -> Session:
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_link(url: str):
    response = get_session().get(url, json=payload)
    result = response.json()
    return float(result["url"].split("/")[-1])


def download_all(urls: list) -> None:
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        responses = {executor.submit(download_link, row): row for row in urls}
        results.extend([future.result()] for future in concurrent.futures.as_completed(responses))
    return results

thread_local = local()

if __name__ == "__main__":
    for ntimes in tries:
        start_time = time.time()
        futures = download_all(urls[:ntimes])
        end_time = time.time()
        logging.info(f"Fetch {ntimes} urls takes {end_time - start_time} seconds")
        logging.info(f"{len(list(futures))} urls were read successfully")
        logging.info(f"conctent: {list(futures)}")


In [ ]:
import asyncio
import time
import aiohttp
from aiohttp.client import ClientSession
# --------------only in jupyter notebook--------------
import nest_asyncio  # only in jupyter notebook
nest_asyncio.apply()


async def download_link(url: str, session: ClientSession):
    async with session.get(url, json=payload) as response:
        result = await response.json()
        req_time = float(result["url"].split("/")[-1])
        return req_time


async def download_all(urls: list):
    my_conn = aiohttp.TCPConnector(limit=100)
    tasks = []
    async with aiohttp.ClientSession(connector=my_conn) as session:
        for url in urls:
            task = asyncio.ensure_future(download_link(url=url, session=session))
            tasks.append(task)
        responses = await asyncio.gather(*tasks, return_exceptions=True)  # the await must be nest inside of the session
    return [response for response in responses]

if __name__ == "__main__":
    for ntimes in tries:
        start_time = time.time()
        futures = asyncio.run(download_all(urls[:ntimes]))
        end_time = time.time()
        logging.info(f"Fetch {ntimes} urls takes {end_time - start_time} seconds")
        logging.info(f"{len(list(futures))} urls were read successfully")
        logging.info(f"conctent: {list(futures)}")
